In [1]:
import gymnasium
from gymnasium import spaces
import numpy as np
import pandas as pd
import heapq
import matplotlib.pyplot as plt
from collections import defaultdict

class RadarTaskSchedulerEnv(gymnasium.Env):
    metadata = {"render_modes": ["human"], "render_fps": 1}

    def __init__(self, task_file="dataset2.csv", render_mode=None):
        super(RadarTaskSchedulerEnv, self).__init__()

        self.tasks_df = pd.read_csv(task_file)
        self.num_tasks = len(self.tasks_df)

        self.priority_queue = []
        self.entry_count = 0

        self.current_task_data = None

        self.scheduled_tasks_log = []

        self.observation_space = spaces.Box(low=0, high=100, shape=(3,), dtype=np.int64)

        self.action_space = spaces.MultiDiscrete([3, 3, 3])

        self.render_mode = render_mode

        self._last_info = {}
        self._last_reward = 0

    def reset(self,*, seed=None, options=None):
        super().reset(seed=seed)

        self.priority_queue = []
        self.entry_count = 0

        for index, row in self.tasks_df.iterrows():
            negative_priority = -row['Priority']
            heapq.heappush(self.priority_queue, (negative_priority, self.entry_count, row.to_dict()))
            self.entry_count += 1
        
        self.scheduled_tasks_log = []
        self._last_info = {}
        self._last_reward = 0

        obs = self._get_observation()
        
        return obs, {}

    def step(self, action):
        if not self.priority_queue:
            return np.array([0, 0, 0], dtype=np.int64), 0, True, False, {"task_dropped": False, "task_id": "N/A", "message": "No tasks left to process."}

        _priority, _entry_count, task_to_process = heapq.heappop(self.priority_queue)
        
        delay, compress, radar_choice = action
        
        # Calculate TNS (Task Nominal Start Time)
        calculated_tns = task_to_process['Request_Time'] + delay

        # Calculate Pn (Power Nominal) based on the compress action
        power_increase_from_compression = 0.0
        if compress == 0:
            power_increase_from_compression = 0.0
        elif compress == 1:
            power_increase_from_compression = 0.15 * task_to_process['Init_Power']
        elif compress == 2:
            power_increase_from_compression = 0.30 * task_to_process['Init_Power']

        calculated_Pn = task_to_process['Init_Power'] + power_increase_from_compression

        # Check for task drop conditions
        task_dropped = False
        if (calculated_tns + task_to_process['Duration']) > task_to_process['Deadline']:
            task_dropped = True
        if calculated_Pn > task_to_process['Max_Power']:
            task_dropped = True

        reward = 0.0

        if not task_dropped:
            reward += 1.0
        else:
            reward -= 1.0

        if not task_dropped:
            time_to_deadline = task_to_process['Deadline'] - (calculated_tns + task_to_process['Duration'])
            if time_to_deadline < 2:
                reward -= 0.1
            elif time_to_deadline < 0:
                reward -= 0.2
        elif (calculated_tns + task_to_process['Duration']) > task_to_process['Deadline'] + 5:
             reward -= 0.2

        if not task_dropped:
            power_margin = task_to_process['Max_Power'] - calculated_Pn
            if power_margin < 2:
                reward -= 0.1
            elif power_margin < 0:
                reward -= 0.2
        elif calculated_Pn > task_to_process['Max_Power'] + 5:
            reward -= 0.2

        if delay == 0 and not task_dropped:
            reward += 0.05

        if compress == 0 and not task_dropped:
            reward += 0.05

        info = { 
            "task_id": task_to_process["Task_ID"],
            "task_dropped": task_dropped,
            "delay_action": delay,
            "compress_action": compress,
            "radar_choice_action": radar_choice,
            "calculated_tns": calculated_tns,
            "calculated_Pn": calculated_Pn,
            "original_deadline": task_to_process['Deadline'],
            "original_max_power": task_to_process['Max_Power']
        }
        self._last_info = info
        self._last_reward = reward

        self.scheduled_tasks_log.append({
            'Task_ID': task_to_process['Task_ID'],
            'Priority': task_to_process['Priority'],
            'Action_Delay': delay,
            'Action_Compress': compress,
            'Action_Radar': radar_choice,
            'Dropped': task_dropped,
            'Final_TNS': calculated_tns,
            'Final_Pn': calculated_Pn,
            'Original_Deadline': task_to_process['Deadline'],
            'Original_Max_Power': task_to_process['Max_Power']
        })

        done = not self.priority_queue

        next_obs = self._get_observation()
            
        return next_obs, reward, done, False, info

    def _get_observation(self):
        if self.priority_queue:
            _priority, _entry_count, task_dict = self.priority_queue[0] 
            self.current_task_data = task_dict
            return np.array([int(task_dict['Duration']), int(task_dict['Deadline']), int(task_dict['Init_Power'])], dtype=np.int64)
        else:
            self.current_task_data = None
            return np.array([0, 0, 0], dtype=np.int64)

    def close(self):
        pass

class QLearningAgent:
    def __init__(self, state_size, action_size, alpha, gamma, epsilon, epsilon_decay, final_epsilon):
        self.q_table = defaultdict(lambda: np.zeros(action_size))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.initial_epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon
        self.action_size = action_size
        self.state_size = state_size

    def choose_action(self, state):
        state_key = str(state)
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.action_size)
        return np.argmax(self.q_table[state_key])

    def update_q_value(self, state, action, reward, next_state):
        state_key = str(state)
        next_state_key = str(next_state)

        best_next_action = np.argmax(self.q_table[next_state_key])
        
        td_target = reward + self.gamma * self.q_table[next_state_key][best_next_action]
        
        td_error = td_target - self.q_table[state_key][action]
        
        self.q_table[state_key][action] += self.alpha * td_error

    def decay_epsilon(self):
        self.epsilon = max(self.epsilon * self.epsilon_decay, self.final_epsilon)

if __name__ == "__main__":
    render_mode_choice = None
    print("Pygame rendering is DISABLED by default in this version.")

    env = RadarTaskSchedulerEnv(task_file="dataset2.csv", render_mode=render_mode_choice)
    
    agent = QLearningAgent(
        state_size=3,
        action_size=27,
        alpha=0.15,
        gamma=0.95,
        epsilon=0.9,
        epsilon_decay=0.99,
        final_epsilon=0.05
    )

    def decode_action(index):
        delay = index // 9
        compress = (index % 9) // 3
        radar = index % 3
        return [delay, compress, radar]

    episodes = 1000
    reward_log = []
    task_drop_log = []

    output_filename = "q_learning_results_no_render.txt"
    with open(output_filename, "w") as f_out:
        f_out.write("--- Q-Learning Training Log (No Rendering) ---\n\n")

        print("Starting Q-Learning training...")
        f_out.write("Starting Q-Learning training...\n\n")

        for episode in range(episodes):
            state, info = env.reset()
            done = False
            total_reward = 0
            task_drops = 0
            episode_step_count = 0

            while not done:
                action_index = agent.choose_action(state)
                action = decode_action(action_index)
                
                next_state, reward, done, _, info = env.step(action)
                
                if env.current_task_data is not None or episode_step_count > 0:
                    agent.update_q_value(state, action_index, reward, next_state)

                state = next_state
                total_reward += reward
                episode_step_count += 1

                if info.get('task_dropped', False):
                    task_drops += 1
            
            reward_log.append(total_reward)
            task_drop_log.append(task_drops)

            agent.decay_epsilon()

            if (episode + 1) % 50 == 0 or episode == episodes - 1:
                log_message = f"Episode {episode+1}/{episodes} — Tasks Processed: {episode_step_count} — Total Reward: {total_reward:.2f} — Tasks Dropped: {task_drops} — Epsilon: {agent.epsilon:.3f}"
                print(log_message)
                f_out.write(log_message + "\n")

                f_out.write(f"\n--- Q-Table after Episode {episode+1} (first 10 entries) ---\n")
                q_table_entries_printed = 0
                for s_key, a_values in agent.q_table.items():
                    f_out.write(f"State: {s_key}\n")
                    action_values_str = ", ".join([f"{val:.2f}" for val in a_values])
                    f_out.write(f"  Q-values: [{action_values_str}]\n")
                    q_table_entries_printed += 1
                    if q_table_entries_printed >= 10:
                        f_out.write("  ...\n")
                        break
                f_out.write("---\n\n")

    print("\nTraining complete!")
    with open(output_filename, "a") as f_out:
        f_out.write("\nTraining complete!\n")

        window_size = 50

        if not reward_log:
            print("\nReward log is empty. No reward plot will be generated.")
            f_out.write("\nReward log is empty. No reward plot will be generated.\n")
        else:
            moving_avg = []
            x_moving_avg = []
            if len(reward_log) >= window_size:
                moving_avg = np.convolve(reward_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg = range(window_size - 1, len(reward_log))

            plt.figure(figsize=(14,7))
            plt.plot(reward_log, label='Episode Rewards', color='orange', alpha=0.7, linewidth=1)
            
            if moving_avg.size > 0:
                plt.plot(x_moving_avg, moving_avg, label=f'Moving Average ({window_size} episodes)', color='blue', linewidth=2)
            
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Total Reward', fontsize=12)
            plt.title('Total Reward over Episodes (Q-Learning)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("reward_plot_no_render.png")
            plt.close()
            print(f"Reward plot saved as reward_plot_no_render.png")
            f_out.write(f"Reward plot saved as reward_plot_no_render.png\n")

        if not task_drop_log:
            print("\nTask drop log is empty. No task drop plot will be generated.")
            f_out.write("\nTask drop log is empty. No task drop plot will be generated.\n")
        else:
            moving_avg_drops = []
            x_moving_avg_drops = []
            if len(task_drop_log) >= window_size:
                moving_avg_drops = np.convolve(task_drop_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg_drops = range(window_size - 1, len(task_drop_log))

            plt.figure(figsize=(14,7))
            plt.plot(task_drop_log, label='Tasks Dropped per Episode', color='red', alpha=0.8, linewidth=1)
            
            if moving_avg_drops.size > 0:
                plt.plot(x_moving_avg_drops, moving_avg_drops, label=f'Moving Average ({window_size} episodes)', color='darkred', linewidth=2)
            
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Tasks Dropped', fontsize=12)
            plt.title('Number of Tasks Dropped per Episode', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("task_drop_plot_no_render.png")
            plt.close()
            print(f"Task drop plot saved as task_drop_plot_no_render.png")
            f_out.write(f"Task drop plot saved as task_drop_plot_no_render.png\n")

        f_out.write("\n\n--- Final Task Schedule (from the last completed training episode) ---\n")
        print("\n--- Final Task Schedule (from the last completed training episode) ---")
        if env.scheduled_tasks_log and len(env.scheduled_tasks_log) == env.num_tasks:
            schedule_df = pd.DataFrame(env.scheduled_tasks_log)
            f_out.write(schedule_df.to_string(index=False))
            print(schedule_df.to_string(index=False))
        else:
            f_out.write("The last episode might have been interrupted or did not complete all tasks.\n")
            f_out.write("To see a full schedule, ensure the training completes at least one full episode.\n")
            print("The last episode might have been interrupted or did not complete all tasks.")
            print("To see a full schedule, ensure the training completes at least one full episode.")
        
        f_out.write("\n\n--- Performance Metrics ---\n")
        print("\n--- Performance Metrics ---")

        if env.scheduled_tasks_log:
            total_tasks_attempted = len(env.scheduled_tasks_log)
            completed_tasks = sum(1 for task in env.scheduled_tasks_log if not task['Dropped'])
            dropped_tasks_count = sum(1 for task in env.scheduled_tasks_log if task['Dropped'])

            if total_tasks_attempted > 0:
                percent_tasks_completed = (completed_tasks / total_tasks_attempted) * 100
                percent_tasks_dropped = (dropped_tasks_count / total_tasks_attempted) * 100
            else:
                percent_tasks_completed = 0.0
                percent_tasks_dropped = 0.0

            metrics = {
                "Total Tasks in Dataset": env.num_tasks,
                "Tasks Attempted (in last episode)": total_tasks_attempted,
                "Tasks Successfully Completed": completed_tasks,
                "Tasks Dropped": dropped_tasks_count,
                "Percentage of Tasks Completed": f"{percent_tasks_completed:.2f}%",
                "Percentage of Tasks Dropped": f"{percent_tasks_dropped:.2f}%"
            }

            metrics_df = pd.DataFrame(metrics.items(), columns=['Metric', 'Value'])
            f_out.write(metrics_df.to_string(index=False))
            print(metrics_df.to_string(index=False))
        else:
            f_out.write("No tasks were processed in the last completed episode to calculate metrics.\n")
            print("No tasks were processed in the last completed episode to calculate metrics.")

    env.close()
    print("Environment closed.")

Pygame rendering is DISABLED by default in this version.
Starting Q-Learning training...
Episode 50/1000 — Tasks Processed: 100 — Total Reward: 64.15 — Tasks Dropped: 18 — Epsilon: 0.545
Episode 100/1000 — Tasks Processed: 100 — Total Reward: 80.60 — Tasks Dropped: 11 — Epsilon: 0.329
Episode 150/1000 — Tasks Processed: 100 — Total Reward: 93.95 — Tasks Dropped: 4 — Epsilon: 0.199
Episode 200/1000 — Tasks Processed: 100 — Total Reward: 98.95 — Tasks Dropped: 2 — Epsilon: 0.121
Episode 250/1000 — Tasks Processed: 100 — Total Reward: 103.30 — Tasks Dropped: 0 — Epsilon: 0.073
Episode 300/1000 — Tasks Processed: 100 — Total Reward: 101.30 — Tasks Dropped: 1 — Epsilon: 0.050
Episode 350/1000 — Tasks Processed: 100 — Total Reward: 101.20 — Tasks Dropped: 1 — Epsilon: 0.050
Episode 400/1000 — Tasks Processed: 100 — Total Reward: 99.05 — Tasks Dropped: 2 — Epsilon: 0.050
Episode 450/1000 — Tasks Processed: 100 — Total Reward: 101.15 — Tasks Dropped: 1 — Epsilon: 0.050
Episode 500/1000 — Tasks